In [0]:
from pyspark.sql.functions import lower,trim,regexp_replace,col,first
from pyspark.sql.types import DecimalType

In [0]:
%run "/Workspace/data_engineering/utils/nb_pmrs_utils_config"

In [0]:
dbutils.widgets.removeAll()

external_location               = dbutils.widgets.text("external_location","")
uc_catalog_name                 = dbutils.widgets.text("uc_catalog_name","")
object_owner_spn                = dbutils.widgets.text("object_owner_spn","")
debug_flag                      = dbutils.widgets.text("debug_flag","")
fact_table                      = dbutils.widgets.text("fact_table","")

external_location               = dbutils.widgets.get("external_location")
uc_catalog_name                 = dbutils.widgets.get("uc_catalog_name")
object_owner_spn                = dbutils.widgets.get("object_owner_spn")
debug_flag                      = dbutils.widgets.get("debug_flag")
fact_table                      = dbutils.widgets.get("fact_table")

In [0]:
try:
    # Get the absolute path
    absolute_path = return_absolute_path(external_location)
    fact_table_rpt_delta_path = f"{absolute_path}data_engineering/gold/finance/{fact_table}_reporting"

    # Debugging information
    if debug_flag == "1":

        print(f"absolute_path        : {absolute_path}")
        print(f"external_location    : {external_location}")
        print(f"object_owner_spn     : {object_owner_spn}")
        print(f"uc_catalog_name      : {uc_catalog_name}")
        print(f"fact_table           : {fact_table}")


except Exception as e:
    error = str(e).replace("'", "")
    raise Exception(f"Error occurred: {error}")

In [0]:
try:
    
    df_measure = spark.sql(f"select * from {uc_catalog_name}.gold_master_data.measure")
    replacements  = [("%", "percent"),("٪", "percent"),("/", "_"),("&","_"),("[- ]", "_"),("_+", "_"),("–", " "),("[^a-zA-Z0-9_]", ""),(r"^_|_$", "")]

    df_measure    = df_measure.withColumn("temp_description", lower(trim(df_measure["measure_description"])))

    # Apply transformations using a loop
    for pattern, replacement in replacements:
        df_measure = df_measure.withColumn("temp_description", regexp_replace(df_measure["temp_description"], pattern, replacement))

    # Rename column
    df_measure  = df_measure.withColumnRenamed("temp_description", "measure_flat_description")
    df_measure  = df_measure.withColumnRenamed("measure_sk", "dim_measure_sk")

except Exception as e:
    error = str(e).replace("'", "")
    raise Exception(f"Error occurred: {error}")

In [0]:
try:
    df_fact = spark.sql(f"select * from {uc_catalog_name}.gold_finance.{fact_table}")
    fact_columns = df_fact.columns
        
    key_columns = [col for col in fact_columns if col not in ['measure_sk', 'value', 'log_id', 'created_date']]

    # Display key columns
    print("Key columns:", key_columns)

    df_joined = df_fact.join(df_measure, df_measure['dim_measure_sk'] == df_fact['measure_sk'], "left").select(*key_columns,'measure_flat_description','value')

    df_unpivoted = df_joined.groupBy(*key_columns).pivot("measure_flat_description").agg(first("value"))

    for column in df_unpivoted.columns:
        if column not in [*key_columns]:
            df_unpivoted = df_unpivoted.withColumn(column, regexp_replace(col(column), "[$,€£¥%]", "").cast(DecimalType(36, 16)))

    if fact_table == "fact_actuals_map":
        fact_tbl = "fact_actuals"
    elif fact_table == "fact_forecast_map":
        fact_tbl = "fact_forecast"
    else:
        fact_tbl = fact_table

    fact_table_rpt_tbl_name = f"{fact_tbl}_reporting"

    concurrent_external_table_delta_write(df_unpivoted, fact_table_rpt_delta_path, 'gold_finance', fact_table_rpt_tbl_name, None, uc_catalog_name, object_owner_spn, retry_count=0,  mergeSchema_flag="False" , delta_overwrite_mode = "full" )

except Exception as e:
    error = str(e).replace("'", "")
    raise Exception(f"Error occurred: {error}")

In [0]:
try:
    
    df_tbl = spark.table(f"{uc_catalog_name}.gold_finance.{fact_tbl}_reporting")

    fact_columns = df_tbl.columns

    # Filter the measure table based on the extracted column names
    filtered_measure_table = df_measure.filter(col("measure_flat_description").isin(fact_columns))

    # Create a dictionary for mapping measure_flat_description to measure_description
    mapping_dict = {row["measure_flat_description"]: row["measure_description"] for row in filtered_measure_table.collect()}

    key_columns = [col for col in fact_columns if col not in mapping_dict]

    # Generate the SELECT statement with aliased columns and key columns
    select_expr = ", ".join([f"`{old_name}` AS `{mapping_dict[old_name]}`" for old_name in fact_columns if old_name in mapping_dict])

    key_columns_expr = ", ".join([
        f"CAST(`{col}` AS INT) AS `{col.replace('_', ' ')}`" if col.lower().endswith('_sk') else f"`{col}` AS `{col.replace('_', ' ')}`"
        for col in key_columns])

    # Combine key columns and aliased columns
    final_select_expr = f"{key_columns_expr}, {select_expr}"

    create_view_statement = f"""
    CREATE OR REPLACE VIEW {uc_catalog_name}.gold_finance.vw_{fact_tbl}_reporting AS
    SELECT {final_select_expr}
    FROM {uc_catalog_name}.gold_finance.{fact_tbl}_reporting f
    where f.bu_sk != -1 and f.product_sk != -1
    """
    #create view
    spark.sql(create_view_statement)

    print(f"{uc_catalog_name}.gold_finance.vw_{fact_tbl}_reporting view created successfully")

except Exception as e:
    error = str(e).replace("'", "")
    raise Exception(f"Error occurred: {error}")   